In [ ]:
import ipywidgets
from ipywidgets import interact

In [ ]:
def f(x):
    return x

In [ ]:
interact(f,x=10)

In [ ]:
!which jupyter

In [ ]:
!which jupyter-lab

In [ ]:
!conda list -n base | grep jupy

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import pandas as pd
    
from pathlib import Path #we'll import Path object with `.ls` method added later
from pprint import pprint as pp

import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


# import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn
pn.extension()

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
)

## Set up additional library path

In [ ]:
# Add the utils directory to the search path
SP_ROOT = Path.home()/'Playground/ContextNet'
SP_LIBS = SP_ROOT/'scripts' # to be changed to 'src'
# LIBS_DIR = Path('../src').absolute()
DIRS_TO_ADD = [SP_LIBS]#, LIBS_DIR]
for p in DIRS_TO_ADD:
    assert p.exists()
    
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(f"Added to sys.path: {p}")

# pp(sys.path)
    

In [ ]:
from output_helpers import print_mro as mro, nprint, Path
import SpacenetPath as spp
import spacenet_globals as spg
# from output_helpers import Path #.ls method is added to Path class

## Step 1: Explore your dataset
    

In [ ]:
city = 'vegas'
rgb8_dir = spp.sample_rgb8_dirs[city]
mask_dir = spp.sample_mask_dirs[city]
sp_vec_dir = spp.sample_road_vec_dirs[city]
osm_mask_dir = spp.sample_mask_dirs[city]

In [ ]:
rgb_fns = sorted([rgb8_dir/fn for fn in rgb8_dir.ls() if Path(fn).suffix in ['.tif', '.tiff']])
mask_fns = sorted([mask_dir/fn for fn in mask_dir.ls() if Path(fn).suffix in ['.tif', '.tiff']])

In [ ]:
for rgb_fn, mask_fn in zip(rgb_fns, mask_fns):
    assert rgb_fn.exists() and mask_fn.exists()
    



## Playground with `param` library

In [ ]:
import xarray as xr
import osmnx as ox
import rasterio as rio
import geopandas as gpd
from rasterio.plot import reshape_as_image

In [ ]:
rgb = xr.open_rasterio(rgb_fns[0])

In [ ]:
rgb_arr = reshape_as_image(rgb.data)
# plt.imshow(rgb_arr)

In [ ]:
basemap = hv.element.tiles.tile_sources['EsriImagery']()

class OSMDL(pm.Parameterized):
    fn = pm.FileSelector(path=str(rgb8_dir/"*.tif"))
   
    osm_log = pm.String(default="", 
                        label="OSM log",
                        doc="simple log string about OSM download status")
    osm_dl_count = pm.Number(0)#, precedence=0) #inivisible widget
    
    action_dl = pm.Action(lambda x: x.param.trigger('action_dl'), label='click to download osm')
    action_click = pm.Action(lambda x: x.param.trigger('action_click'))#, precedence=-1)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.osm_g = None
        self.osm_edges = gpd.GeoDataFrame()
        print('initialized')
    
    @param.depends('action_dl', watch=True)
    def _download_osm(self):
        print('Started downloading osm data')
        with rio.open(self.fn) as ds:
            bounds = ds.bounds
        north, south, east, west = bounds.top, bounds.bottom, bounds.right, bounds.left
        self.osm_g = ox.graph_from_bbox(north, south, east, west)
        self.osm_edges = ox.graph_to_gdfs(self.osm_g, edges=True, nodes=False)
                                                      
        print("OSM data downloaded")
        self.osm_dl_count += 1
        self.osm_log = f'OSM data downloaded: {bounds}, {len(self.osm_edges)}'
        
    def show_osm_edges(self):
        return basemap * gv.Path(self.osm_edges)
    
                           
                           
                           
     

In [ ]:
ex = OSMDL()
col = pn.Column()
for p in ex.param:
    col.append(pn.panel(ex.param[p]))
col.servable()
# pn.panel(ex.param.).servable()

In [ ]:
pn.panel(ex.param.osm_dl_count)


## Example of `param.Action` parameter

In [ ]:
class ActionExample(param.Parameterized):
    """
    Demonstrates how to use param.Action to trigger an update.
    """

    number = param.Integer(default=0)
    
    action = param.Action(lambda x: x.param.trigger('action'), label='Download OSM!')
        
    @param.depends('action')
    def get_number(self):
        print('action triggered')
        self.number += 1
#         return self.number
# todo add init function and see if it resolves this issue
# Then, ask questions about when the parameters stay as class variables vs. instance variable
# - link to the Topica manual

In [ ]:
action_example = ActionExample()
                                
pn.Column(pn.panel(action_example.param),
            'Click the button to trigger an update in the output.',
                 pn.WidgetBox(action_example.get_number, width=300)
).servable()

In [ ]:
pn.panel(action_example.param.number)

In [ ]:
pn.Pane(action_example.param.number)

In [ ]:
class Example(param.Parameterized):
    fn = param.FileSelector(path=str(rgb8_dir/"*.tif"))
    action = param.Action(lambda x: x.param.trigger('action'),
                       label='Download OSM')
    
    osm_count = param.Number(default=0.)#, precedence=0) #inivisible widget
    
    def __init__(self,**kwargs):
#         pdb.set_trace()
        super().__init__(**kwargs)
        print('created object')
    
    @param.depends('action', watch=True)
    def update_osm(self):
        print('Started downloading osm data')
        print("OSM data downloaded")
        self.osm_count += 1
#         self.osm = ...


In [ ]:
ex = Example()

In [ ]:
ex.update_osm()
print(ex.osm_count)

In [ ]:
pn.Column(ex.param)

In [ ]:
pn.panel(ex.param.osm_count)

In [ ]:
ex.osm_count

In [ ]:
ex.param['action']

## Template 

In [ ]:
CITIES = ['LA', 'BOSTON', 'PARIS', 'JEONJU', 'DC', 'SF']
class MyParamedClass(pm.Parameterized):
    
    # Declare parameters (which will turned into instance attributes via. pm.Parametrized.__init__ method)
    age = pm.Integer(10, bounds=(1,100))
    city = pm.Selector(objects=CITIES, default='LA')
    logbox = pm.String('LOGBOX')
    
    # Explicitly define initialization method
    ## This will be called to instantiate new instances for this class
    def __init__(self, **params):
        """
        The signature of this init method matched pm.Parametrized class's init method:
            pm.Parametrized.__init__(self, **params)
        This allows to specify parameter values at instaniation using 'key'=value: 
            eg: me = MyParamedClass(age=11, city='JEONJU')
            
        See pm.Parametrized?? for details
        """
        super().__init__(**params)
        self.country = 'USA'
        
    # Annotate a method with its dependencies that does not need automatic update when the dependents' values change
    @param.depends('age', watch=False) # watch=False is default
    def get_stage(self):
        stage = 'old' if self.age > 50 else 'young'
        return stage
    
    # Annotate method that needs be updated automatically as its dependents' parameter values change
    @param.depends('city', watch=True)
    def update_country(self):
        if self.city in ['LA', 'BOSTON', 'DC', 'SF']:
            self.country = 'USA'
        elif self.city == 'PARIS':
            self.country = 'FRANCE'
        else:
            self.country = 'KOREA'
        print(f"Country is updated: {self.country}")
        
        # Just for demonstrating that this method is actually called when `city` parameter changes
        self.logbox = self.country
        print('Logbox: ', self.logbox)
        
                
    
    
        
    

In [ ]:
myex = MyParamedClass()

In [ ]:
pn.panel(myex.param)#.servable()

In [ ]:
pn.panel(myex.param['logbox'])

## Integrated Dataset Explorer


In [ ]:

class DSExplorer(pm.Parameterized):
    rgb_fn = pm.Selector(rgb_fns)
    mask_alpha = pm.Magnitude(0.5)
    show_legend = pm.Boolean(False)
#     show_rgb_hover = pm.Boolean(False)
#     show_mask_hover = pm.Boolean(True)
    
    # OSM download parameters
    osm_log = pm.String(default="", 
                        label="OSM log",
                        doc="simple log string about OSM download status")
    osm_dl_count = pm.Number(0)#, precedence=0) #inivisible widget
    
    action_dl = pm.Action(lambda x: x.param.trigger('action_dl'), label='click to download osm')
    
    def __init__(self, **params):
        super().__init__(**params)
        self.osm_g = None
        self.osm_edges = gpd.GeoDataFrame()
        print('initialized')
    
    ################################################################################
    # Methods
    ################################################################################
    @pm.depends('rgb_fn', watch=True)
    def get_gv_rgb(self):
        rgb_da = xr.open_rasterio(self.rgb_fn)/255.
        r,g,b = map(np.asarray,[rgb_da.sel(band=1), rgb_da.sel(band=2), rgb_da.sel(band=3)])
        xs, ys = np.array(rgb_da.coords['x']), np.array(rgb_da.coords['y'])
        # gv.RGB(rgb_da, kdims=['x','y'], vdims='R G B'.split()) #fails
        return gv.RGB((xs,ys,r,g,b), 
                        kdims=['Longitude', 'Latitude'], 
                        vdims='R G B'.split(), 
                        crs=ccrs.PlateCarree(),
                       group='rgb')
    
    @pm.depends('rgb_fn', watch=True)
    def get_gv_mask(self):
        fn = get_sp_mask1300_fn(self.rgb_fn)
        da = xr.open_rasterio(fn)/255.
        return gv.Image(da,
                        kdims=['x','y'],  
                        crs=ccrs.PlateCarree(), 
                        group='mask').redim(z='RT')
    
    @pm.depends('rgb_fn', watch=True)
    def get_bounds(self):
        ds = rio.open(self.rgb_fn)
        xmin, ymin, xmax, ymax = ds.bounds
        return hv.Div(
            f""" 
            <h2>Bounds</h2>
            <p>lon:{xmin, xmax},</p>
            <p>lat: {ymin, ymax}</p>""")
    
    @param.depends('action_dl', watch=True)
    def _download_osm(self):
        print('Started downloading osm data')
        with rio.open(self.fn) as ds:
            bounds = ds.bounds
        north, south, east, west = bounds.top, bounds.bottom, bounds.right, bounds.left
        self.osm_g = ox.graph_from_bbox(north, south, east, west)
        self.osm_edges = ox.graph_to_gdfs(self.osm_g, edges=True, nodes=False)
                                                      
        print("OSM data downloaded")
        self.osm_dl_count += 1
        self.osm_log = f'OSM data downloaded: {bounds}, {len(self.osm_edges)}'
        return gv.Path(self.osm_edges)

    def get_gv_osm(self):
        return gv.Path(self.osm_edges)
        

    ################################################################################
    # Viewable
    ################################################################################
    def viewable(self):
        dmap_rgb = hv.DynamicMap(self.get_gv_rgb).opts(show_legend=False)
        dmap_mask = hv.DynamicMap(self.get_gv_mask)
        dmap_bounds = hv.DynamicMap(self.get_bounds)
#         dmap_osm = hv.DynamicMap(self._ßdownload_osm)
        return dmap_rgb * dmap_mask + dmap_bounds #+ dmap_osm

In [ ]:
ex = DSExplorer()
col = pn.Column()
for p in ex.param:
    col.append(pn.panel(ex.param[p]))
pn.Row(col, ex.viewable).servable()

In [ ]:
/Users/hayley/Workspace/Talks/PyData-LA-2019/assets/

Holoviz is a set of python libraries that offers simple yet powerful visualization and GUI tools. While building a model to solve a specific task at hand (eg. predicting the best time to sell a house, discriminating fake news, detecting fires from satellite images), we can use these tools to create simple "playgrounds" such as reactive plots and GUIs to explore our datasets and experiment with various model configurations in intuitive ways. Since HoloViz integrates seamlessly with the notebook environment, it is especially useful during the exploratory and analyzing stages. This tutorial thus focuses on the iterative and experimental nature of modeling, and demonstrates various ways to integrate these tools into four core components of modeling: (1) data exploration, (2) train hyperparameter tuning, (3) monitoring model training, and (4) understanding the trained model beyond a single metric. During this tutorial, we will use the multi-class road detection problem as a concreate example. 




his tutorial focuses on the experimental nature of modeling, and guides through how HoloViz tools can help you create your own little playgrounds to 


and thus enables us to (1)be more experimental and creative during exploratory stages, and (2) gain deeper understanding of the model via direct control over hyperparameter configurations 






naturally iterative and experimental environmnetare powerful in exploratory or analysis stages of modeling.  that is, seamlessly integrated into the rest of your 





During this tutorial, you will learn how to turn your static plotting codes (eg. `plt.plot`, `plt.imshow`) into interactive rich objects, and use the skill to 




This tutorial will introduce various ways to use HoloViz (in conjunction with other libraries like GeoPandas, Numpy and PyTorch) to turn your static plotting codes (eg. using Matplotlib)achieve richer understanding of your datasets and controllable experimentations during modeling. d


to embellish each step of your modeling process, especially when the goal is to  experimental nature of modeling process and introduces various ways to use HoloViz (in conjunction with other Python libraries such as GeoPandas, Numpy and PyTorch) to bring out the 

integrates the exploratory part of your modeling process seamlessly into the rest of workflow in a Jupyter notebook. 

## Overview 
This tutorial introduces various ways to make your data exploration and modeling process more interactive and exploratory by using the combination of `JupyterLab`, `HoloViz` and `PyTorch`.  
During the initial exploratory stage, 

Holoviz is a set of python libraries that offers simple yet powerful visualization and GUI tools. We can use these tools to create micro "playgrounds" within the Jupyter notebook, such as reactive plots and GUIs, to explore the datasets and experiment with different model configurations in a more intuitive way. For example, in [Figure 1](#link to google drive), the rich visualization object from `HoloViz`  shows the Latitude and Longitude values as we hover over the figure, shows the image at different zoom levels and can even do user-specified computations such as calculating a road length in a (mouse) selected area. 


This tutorial focuses on the iterative and experimental nature of modeling, and shows different ways to integrate these tools into four core components of modeling: (1) data exploration, (2) train hyperparameter tuning, (3) monitoring model training, and (4) understanding the trained model beyond a single metric. During this tutorial, we will use the multi-class road detection problem as a concreate example. 





focuses on the experimental nature of modeling and integrates seamlessly with Jupyter notebooks.






I will start by introducing the four core HoloViz libraries (Holoviews, GeoViews, Panel and Param) and demonstrate basic examples on how we can essentially replace any "Matplotlib.pyplot" calls with the equivalents in `HoloViz`.  You will see how this opens up the possibilities to directly interact with your visualization by eg. hovering over the graph to inspect values, querying RGB values of an image on the figure, or querying Lat/Lon values on your map.
